In [ ]:
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.runnables import RunnableConfig
from typing import Annotated
from typing_extensions import TypedDict
from operator import add

class State(TypedDict):
    foo: str
    bar: Annotated[list[str], add]

def node_a(state: State):
    return {"foo": "a", "bar": ["a"]}

def node_b(state: State):
    return {"foo": "b", "bar": ["b"]}


workflow = StateGraph(State)
workflow.add_node(node_a)
workflow.add_node(node_b)
workflow.add_edge(START, "node_a")
workflow.add_edge("node_a", "node_b")
workflow.add_edge("node_b", END)

checkpointer = InMemorySaver()
graph = workflow.compile(checkpointer=checkpointer)

config: RunnableConfig = {"configurable": {"thread_id": "1"}}

In [8]:
graph.get_state(config)

StateSnapshot(values={}, next=(), config={'configurable': {'thread_id': '1'}}, metadata=None, created_at=None, parent_config=None, tasks=(), interrupts=())

In [9]:
graph.invoke({"foo": ""}, config)

{'foo': 'b', 'bar': ['a', 'b']}

In [10]:
graph.get_state(config)

StateSnapshot(values={'foo': 'b', 'bar': ['a', 'b']}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0c67eb-f552-6226-8002-cc4e42025689'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-11-21T02:06:52.669178+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0c67eb-f550-67fd-8001-db21a679f3e5'}}, tasks=(), interrupts=())

In [13]:
list(graph.get_state_history(config))


[StateSnapshot(values={'foo': 'b', 'bar': ['a', 'b']}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0c67eb-f552-6226-8002-cc4e42025689'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-11-21T02:06:52.669178+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0c67eb-f550-67fd-8001-db21a679f3e5'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'foo': 'a', 'bar': ['a']}, next=('node_b',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0c67eb-f550-67fd-8001-db21a679f3e5'}}, metadata={'source': 'loop', 'step': 1, 'parents': {}}, created_at='2025-11-21T02:06:52.668507+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0c67eb-f54e-6699-8000-59210cb2ee50'}}, tasks=(PregelTask(id='07fe8de6-427a-ba7e-0b6a-04b9e3dca38f', name='node_b', path=('__pregel_pull', 'node_b'), error=None, interrupts

可以通过下面的方式回到之前的检查点:
```python
config = {
    "configurable": {
        "thread_id": "1",
        "checkpoint_id": "xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx"
    }
}
graph.invoke(None, config=config)
```

In [ ]:
# 更新图的状态
graph.update_state(config, {
    "foo": 2,
    "bar": ["b"]
})

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f0c67f5-b7bc-6714-8003-e8c88ba59cfa'}}

```python
update_state(
    config: RunnableConfig,
    values: dict[str, Any] | Any | None,
    as_node: str | None = None,
    task_id: str | None = None,
) -> RunnableConfig
```
其实还有一个as_node参数，可以指定现在回到哪个节点开始执行

In [18]:
list(graph.get_state_history(config))

[StateSnapshot(values={'foo': 2, 'bar': ['a', 'b', 'b']}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0c67f5-b7bc-6714-8003-e8c88ba59cfa'}}, metadata={'source': 'update', 'step': 3, 'parents': {}}, created_at='2025-11-21T02:11:14.647001+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0c67eb-f552-6226-8002-cc4e42025689'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'foo': 'b', 'bar': ['a', 'b']}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0c67eb-f552-6226-8002-cc4e42025689'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-11-21T02:06:52.669178+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0c67eb-f550-67fd-8001-db21a679f3e5'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'foo': 'a', 'bar': ['a']}, next=('node_b',), config={'configurable': {'thread_id':